In [1]:
import torch

# Local imports
from train import train_model

Using device: cpu


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")

Using device: cuda:0


In [3]:
args = {
    "use_small_dataset": False,
    "batch_size": 64,
    "stoppage_epochs": 42,
    "max_epochs": 512,
    "seed": 42,
    "data_path": "../data",
    "frac_train": 0.8,
    "frac_validation": 0.1,
    "frac_test": 0.1,
    "huber_beta": 2.0,
    "weight_decay": 1e-3,
    "lr": 3e-4,
    "scheduler_patience": 10,
    "scheduler_factor": 0.5,
    "hidden_size": 96,
    "num_layers": 7,
    "num_attn_heads": 6,
    "dropout": 0.2,
    "leaky_relu_slope": 0.2,
    "pooling_dim": 96,
}

In [ ]:
import argparse

training_args = argparse.Namespace(**args)
train_model(training_args, device)

Model parameters: 182714


Epoch 1/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [02:35<00:00,  5.84it/s]


Epoch 1/512: 
Train Loss = 0.32948, Train MSE = 1.33976, Train MAE = 0.94078, Train Acc = 0.59302
Val Loss = 0.32289, Val MSE = 1.31338, Val MAE = 0.93278, Val Acc = 0.60104


Epoch 2/512: 100%|██████████████████████████████████████████████████████████████████████| 910/910 [00:44<00:00, 20.54it/s]


Epoch 2/512: 
Train Loss = 0.32184, Train MSE = 1.30736, Train MAE = 0.92894, Train Acc = 0.59962
Val Loss = 0.31297, Val MSE = 1.27246, Val MAE = 0.91325, Val Acc = 0.61122


...

Epoch 22/512: 100%|█████████████████████████████████████████████████████████████████████| 910/910 [00:44<00:00, 20.44it/s]


Epoch 22/512: 
Train Loss = 0.22214, Train MSE = 0.89857, Train MAE = 0.75112, Train Acc = 0.71630
Val Loss = 0.22157, Val MSE = 0.89820, Val MAE = 0.74858, Val Acc = 0.71584


Epoch 23/512: 100%|█████████████████████████████████████████████████████████████████████| 910/910 [00:44<00:00, 20.28it/s]


Epoch 23/512: 
Train Loss = 0.21929, Train MSE = 0.88651, Train MAE = 0.74575, Train Acc = 0.72251
Val Loss = 0.22724, Val MSE = 0.92335, Val MAE = 0.75456, Val Acc = 0.71144


Epoch 24/512:   1%|▊                                                                     | 10/910 [00:00<00:44, 20.44it/s]